In [0]:
## Task 2 ,Silver layer (Enriching and cleaning of the data)
## 
# Read from Bronze Layer and join with other sources to enrich or filter the data
df_bronze = spark.readStream.format("delta").load("abfss://bronze@storage.dfs.core.windows.net/inventory/")

# Join with supplier data
df_silver = df_bronze.alias("b") \
    .join(spark.read.format("delta").load("abfss://silver@storage.dfs.core.windows.net/win_data/").alias("s"),
          col("s.purchase_order") == col("b.purchase_order"), "left") \
    .select(
        col("b.product_id"), 
        col("b.store_id"), 
        col("b.event_time"),
        when(col("b.event_time").isNotNNUll(),from_utc_timestamp(col("b.event_time"),"EST")).otherwise(None).alias("event_time_est"),
        col("b.quantity"), 
        col("b.temperature"), 
        col("s.supplier_name"),
        col("s.category")
    ) \
    .where(col("b.shipment_id").isNotNull) # Fetching the details of the shipments and vendors that have sensors

# Write to Delta Table (Silver Layer)
df_silver.writeStream \
    .format("delta") \
    .option("checkpointLocation", "abfss://silver@storage.dfs.core.windows.net/checkpoints/") \
    .option("path", "abfss://silver@storage.dfs.core.windows.net/shipments_deltatrak_sensors/") \
    .outputMode("append") \
    .start()